In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
dataset = load_dataset('daily_dialog')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/daily_dialog
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(

In [ ]:
def generate_prompt(examples):
    prompt_list = []
    for dialog in examples['dialog']:
        prompt = "<bos>"
        for idx, utterance in enumerate(dialog):
            role = "user" if idx % 2 == 0 else "model"
            prompt += f"<start_of_turn>{role}\n{utterance.strip()}<end_of_turn>\n"
        prompt += "<eos>"
        prompt_list.append(prompt)
    return {'text': prompt_list}

In [ ]:
tokenized_datasets = dataset.map(generate_prompt, batched=True, remove_columns=['dialog', 'act', 'emotion'])

In [ ]:
model_name = 'google/gemma-2b-it'  # GEMMA 모델의 실제 이름 또는 경로로 변경하세요
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
special_tokens = {
    'bos_token': '<bos>',
    'eos_token': '<eos>',
    'additional_special_tokens': ['<start_of_turn>', '<end_of_turn>', 'user', 'model']
}
tokenizer.add_special_tokens(special_tokens)


0

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto'
)
model.resize_token_embeddings(len(tokenizer))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Embedding(256000, 2048, padding_idx=0)

In [ ]:
!pip install --upgrade peft bitsandbytes



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.1 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.42.0
    Uninstalling bitsandbytes-0.42.0:
      Successfully uninstalled bitsandbytes-0.42.0


In [ ]:
# LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["gate_proj", "up_proj", "down_proj"],  # GemmaMLP의 선형 계층 이름
    lora_dropout=0.2,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 15,925,248 || all params: 2,522,097,664 || trainable%: 0.6314


In [ ]:
print(tokenized_datasets.column_names)


{'train': ['text'], 'validation': ['text'], 'test': ['text']}


In [ ]:
print(tokenized_datasets['train'].column_names)
print(tokenized_datasets['validation'].column_names)
print(tokenized_datasets['test'].column_names)

['text']
['text']
['text']


In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized


In [ ]:
tokenized_datasets = tokenized_datasets.map(tokenize_function, batched=True, remove_columns=['text'])



Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])



In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    num_train_epochs=1,  # 초기 테스트를 위해 에포크 수를 줄였습니다.
    gradient_accumulation_steps=16,
    fp16=True,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    warmup_steps=500,
    lr_scheduler_type="linear"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()
model = model.merge_and_unload()

# 전체 모델과 토크나이저 저장
model.save_pretrained("/content/drive/MyDrive/finetuned_models")
tokenizer.save_pretrained("/content/drive/MyDrive/finetuned_tokenizer")

Step,Training Loss
100,10.689500
200,3.332300
300,2.507400


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('/content/drive/MyDrive/finetuned_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/finetuned_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/finetuned_tokenizer/tokenizer.model',
 '/content/drive/MyDrive/finetuned_tokenizer/added_tokens.json')

In [ ]:
model = model.merge_and_unload()

# 전체 모델과 토크나이저 저장
model.save_pretrained("/content/drive/MyDrive/finetuned_models")
tokenizer.save_pretrained("/content/drive/MyDrive/finetuned_tokenizer")